In [1]:
import pandas as pd

# Define file paths
file_paths = {
    "Nutrient_Values": "2021-2023 FNDDS At A Glance - FNDDS Nutrient Values.xlsx",
    "Ingredient_Nutrient_Values": "2021-2023 FNDDS At A Glance - Ingredient Nutrient Values.xlsx",
    "Portions_and_Weights": "2021-2023 FNDDS At A Glance - Portions and Weights.xlsx",
    "Ingredients": "2021-2023 FNDDS At A Glance - FNDDS Ingredients.xlsx",
    "Foods_and_Beverages": "2021-2023 FNDDS At A Glance - Foods and Beverages.xlsx"
}

# Output file path
output_file = "Merged_FNDDS_Dataset.xlsx"

# Initialize a writer for the Excel file
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet_name, file_path in file_paths.items():
        # Read each Excel file into a DataFrame
        df = pd.read_excel(file_path)
        # Write the DataFrame to a new sheet in the output Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Merged Excel file has been saved to {output_file}.")


Merged Excel file has been saved to Merged_FNDDS_Dataset.xlsx.


In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the merged dataset
merged_file = "Merged_FNDDS_Dataset.xlsx"
nutrient_values = pd.read_excel(merged_file, sheet_name="Nutrient_Values")

# Define macronutrient targets per meal
total_calories = 1600
total_protein = 100
total_fat = 70
total_carbs = 250

meal_targets = {
    "Breakfast": {"calories": total_calories * 0.3, "protein": total_protein * 0.3, "fat": total_fat * 0.3, "carbs": total_carbs * 0.3},
    "Lunch": {"calories": total_calories * 0.4, "protein": total_protein * 0.4, "fat": total_fat * 0.4, "carbs": total_carbs * 0.4},
    "Dinner": {"calories": total_calories * 0.3, "protein": total_protein * 0.3, "fat": total_fat * 0.3, "carbs": total_carbs * 0.3},
}

# Extract relevant columns for optimization
foods = nutrient_values[["Main food description", "Energy (kcal)", "Protein (g)", "Total Fat (g)", "Carbohydrate (g)"]]
foods = foods.dropna()  # Drop rows with missing values

# Objective function to minimize for one food item
def objective(x, food_index, meal_targets):
    quantity = x[0]
    total_calories = quantity * foods.loc[food_index, "Energy (kcal)"] / 100
    total_protein = quantity * foods.loc[food_index, "Protein (g)"] / 100
    total_fat = quantity * foods.loc[food_index, "Total Fat (g)"] / 100
    total_carbs = quantity * foods.loc[food_index, "Carbohydrate (g)"] / 100
    
    return (
        (total_calories - meal_targets["calories"]) ** 2 +
        (total_protein - meal_targets["protein"]) ** 2 +
        (total_fat - meal_targets["fat"]) ** 2 +
        (total_carbs - meal_targets["carbs"]) ** 2
    )

# Run optimization for a single food item
def optimize_food_for_meal(meal_targets, excluded_foods):
    best_food = None
    best_quantity = None
    best_score = float("inf")
    
    for food_index in range(len(foods)):
        food_name = foods.iloc[food_index]["Main food description"]
        if food_name in excluded_foods:
            continue  # Skip already selected foods
        
        # Bounds for quantity: 0 to 500 grams
        bounds = [(0, 500)]
        
        # Initial guess: 100 grams
        initial_guess = [100]
        
        # Minimize the objective function
        result = minimize(
            objective, initial_guess, args=(food_index, meal_targets), bounds=bounds, method="SLSQP"
        )
        
        # Check if the result is better than the current best
        if result.success and result.fun < best_score:
            best_food = foods.iloc[food_index]
            best_quantity = result.x[0]
            best_score = result.fun
    
    return best_food, best_quantity

# Optimize for each meal
optimized_meals = {}
excluded_foods = set()
for meal, targets in meal_targets.items():
    print(f"Optimizing {meal}...")
    best_food, best_quantity = optimize_food_for_meal(targets, excluded_foods)
    if best_food is not None:
        optimized_meals[meal] = {
            "food_name": best_food["Main food description"],
            "quantity": best_quantity,
            "energy": best_food["Energy (kcal)"] * best_quantity / 100,
            "protein": best_food["Protein (g)"] * best_quantity / 100,
            "fat": best_food["Total Fat (g)"] * best_quantity / 100,
            "carbs": best_food["Carbohydrate (g)"] * best_quantity / 100,
        }
        excluded_foods.add(best_food["Main food description"])  # Add the selected food to the exclusion set

# Display optimized meals
print("\nOptimized Meal Plan:")
for meal, details in optimized_meals.items():
    print(f"\n{meal}:")
    print(f"  Food: {details['food_name']}")
    print(f"  Quantity: {details['quantity']:.2f} g")
    print(f"  Energy: {details['energy']:.2f} kcal")
    print(f"  Protein: {details['protein']:.2f} g")
    print(f"  Fat: {details['fat']:.2f} g")
    print(f"  Carbs: {details['carbs']:.2f} g")


Optimizing Breakfast...
Optimizing Lunch...
Optimizing Dinner...

Optimized Meal Plan:

Breakfast:
  Food: Vegetable mixture, dried
  Quantity: 146.61 g
  Energy: 480.88 kcal
  Protein: 25.52 g
  Fat: 13.02 g
  Carbs: 72.16 g

Lunch:
  Food: Chickpeas, from canned, no added fat
  Quantity: 438.88 g
  Energy: 640.76 kcal
  Protein: 35.94 g
  Fat: 14.22 g
  Carbs: 98.53 g

Dinner:
  Food: Pasta with sauce and meat, from school lunch
  Quantity: 411.07 g
  Energy: 480.96 kcal
  Protein: 24.46 g
  Fat: 12.83 g
  Carbs: 71.94 g
